# Agentic RAG

In [10]:
!pip install smolagents
!pip install langchain
!pip install duckduckgo-search
!pip install bs4
!pip install requests
!pip install json

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [7]:
import os
from smolagents import DuckDuckGoSearchTool

search_tool = DuckDuckGoSearchTool()
output = search_tool("Who wrote odyssey poem?")

print(output)

## Search Results

[Odyssey - Wikipedia](https://en.wikipedia.org/wiki/Odyssey)
The Odyssey (/ ˈ ɒ d ɪ s i /; [1] Ancient Greek: Ὀδύσσεια, romanized: Odýsseia) [2] [3] is one of two major epics of ancient Greek literature attributed to Homer.It is one of the oldest surviving works of literature and remains popular with modern audiences. Like the Iliad, the Odyssey is divided into 24 books.It follows the heroic king of Ithaca, Odysseus, also known by the Latin ...

[Odyssey | Summary, Characters, Meaning, & Facts | Britannica](https://www.britannica.com/topic/Odyssey-epic-by-Homer)
Scholars date the writing of the Odyssey to about 725-675 bce.The poem was intended for oral performance. It was composed of 12,109 lines written in dactylic hexameter (sometimes referred to as "Homeric hexameter")—that is, each line consisted of six feet, or metrical units, and each foot consisted of a dactyl (a stressed syllable followed by two unstressed syllables).

[Odyssey: Homer's Epic Poem of Redempti

In [12]:
from langchain.tools import Tool
from bs4 import BeautifulSoup as bs

import requests
import json
import os

def get_vulnerability_report(input: str) -> str:
  """Grab today's vulnerability report from CVE"""
  url = f"https://www.cve.org/CVERecord/SearchResults?query={input}"
  data = requests.get(url)
  soup = bs(data.text, 'html.parser')
  table = soup.find('table')
  return f"{table}"

cve_info_tool = Tool(
    name="get_vulnerability_report",
    description="Grab today's vulnerability report from CVE",
    func=get_vulnerability_report
)